In [2]:
import pandas as pd
import numpy as np
import re


# Reading data
ad_orig = []

# Dictionary to normalize the data
replacements = {
    #";", ","
    '"': "",
    "º": "ND",
    # "º": "",
    "Á": "A",
    "É": "E",
    "Í": "I",
    "Ó": "O",
    "Ú": "U",
    "Ñ": "N",
    "Ü": "U",
    "C/": "CALLE",
    "C /": "CALLE",
    "AVDA": "AVENIDA",
    "CTRA": "CARRETERA",
    "URB. ": "URBANIZACION ",
    "URB " : "URBANIZACION ",
    "URB." : "URBANIZACION ",
    "AVENIDAAVENIDA" : "AVENIDA",
    "\n": "",
    "PLAZAPLAZA": "PLAZA"
}

# Opening and performing alterations
with open('Addresses_Orig_csv.csv', encoding='utf-8') as file:
    for line in file:
        line = line.upper()
        for key, value in replacements.items():
            line = line.replace(key, value)
        
        # Split the line by the semicolon, normally its the neighborhood and province
        line = line.split(';')

        ad_orig.append(line)

# From the list, create a dataframe
ad_orig = pd.DataFrame(ad_orig)
# Create the column names
ad_orig.columns = ["ADDRESSES", "NEIGHBORHOOD", "PROVINCE"]

# Save it as CSV
ad_orig.to_csv('Add_nosc_csv.csv', index=False, header=True, encoding='utf-8')
df = pd.read_csv('Add_nosc_csv.csv')


def type_street(add):
    """
    This function is used to create a new column with the type of street
    in case CALLE or AVENIDA is present in the address
    """
    prefixes = ['CALLE', 'AVENIDA']#, 'CARRETERA']
    # print(type(add))
    for p in prefixes:
        if str(add).startswith(p):
            # print(add.startswith(p))
            # print("TRUE")
            return p
    return ""

def remove_prefix(add):
    """ 
    This function is to remove the prefixes from the address
    in case they are present
    """
    prefixes = ['CALLE', 'AVENIDA']#, 'CARRETERA']
    # print(type(add))
    for p in prefixes:
        if str(add).startswith(p):
            address_parts = str(add).split(maxsplit=1)
            if len(address_parts) > 1:
                return address_parts[1]
    return add



# Create a new column 'Type of street' based on the prefixes
df['TYPE_OF_STREET'] = df['ADDRESSES'].apply(type_street)

# Remove the prefixes from the original 'Street' column
df['ADDRESSES'] = df['ADDRESSES'].apply(remove_prefix)

# Move the 'Type' column to the first position
type_column = df.pop('TYPE_OF_STREET')
df.insert(0, 'TYPE_OF_STREET', type_column)

#drop first line that is empty (is the header of the csv)
df = df.drop(df.index[0])

# Function to rearrange the neighborhood names
def rearrange_name(name):
    """
    This function is to rearrange the neighborhood names.
    Ex: 'MANGA DEL MAR, LA' -> 'LA MANGA DEL MAR'
    """
    if isinstance(name, str) and ',' in name:
        # Split the name into two parts around the comma
        parts = name.split(',')
        # Remove leading/trailing white space and reverse the parts
        parts = [part.strip() for part in reversed(parts)]
        # Join the parts back together with a space
        name = ' '.join(parts)
    return name

# Apply the function to the 'NEIGBORHOOD' column
df['NEIGHBORHOOD'] = df['NEIGHBORHOOD'].apply(rearrange_name)

# Fill NaN values in 'NEIGHBORHOOD' with an empty string in case there are any
df['NEIGHBORHOOD'] = df['NEIGHBORHOOD'].fillna('')

# Change column names
df.columns = ['type_of_street','addresses', 'neighborhood', 'province']

# Space out 'addresses' entries in case they are not spaced out
df['addresses'] = df['addresses'].str.replace('(\d+)', r' \1 ').str.replace('([A-Z]{2,})', r' \1 ')

# Convert all entries to lower case for uniformity
df = df.applymap(lambda s:s.lower() if type(s) == str else s)

# Replace 'NO ENCONTRADA' with np.nan
df.replace('no encontrada', np.nan, inplace=True)

# Drop rows where 'addresses' column is NaN
df.dropna(subset=['addresses'], inplace=True)

def add_space_between_letter_and_number(address):
    """
    This function adds a space between the last letter and the number
    """
    # Define a regular expression pattern to extract numbers preceded by a letter
    pattern = r'(\D)(\d+)'

    if isinstance(address, str):  # Check if the value is a string
        address = re.sub(pattern, r'\1 \2', address)
    return address

# Apply the function to the 'addresses' column and update the values
df['addresses'] = df['addresses'].apply(add_space_between_letter_and_number)

def add_space_between_number_and_letter(address):
    """ 
    This function adds a space between the number and the following non-digit character
    """
    # Define a regular expression pattern to detect a number followed by a letter without a space
    pattern = r'(\d+)(\D)'
    
    if isinstance(address, str):  # Check if the value is a string
        address = re.sub(pattern, r'\1 \2', address)
    return address

# Apply the function to add a space between the number and the following non-digit character
df['addresses'] = df['addresses'].apply(add_space_between_number_and_letter)

# Detect "Urbanizacion" wrong spelled and replace it
def check_urb(address):
    if 'urbanizaci' in address and 'urbanizacion' not in address:
        address = address.replace('urbanizaci', 'urbanizacion ')
    elif 'urbaniz' in address and 'urbanizacion' not in address:
        address = address.replace('urbaniz', 'urbanizacion ')
    elif 'urb' in address and 'urbanizacion' not in address:
        address = address.replace('urb', 'urbanizacion ')
    if 'apartamentoapartamento' in address:
        address = address.replace('apartamentoapartamento', 'apartamento ')
    if 'plaza' in address and 'plaza ' not in address:
        address = address.replace('plaza', 'plaza ')
    if 'duplex' in address and 'duplex ' not in address:
        address = address.replace('duplex', 'duplex ')
    if 'parcelafosforo' in address:
        address = address.replace('parcelafosforo, esquina con ', '')

    return address

# Apply the check_urb function to the 'addresses' column
df['addresses'] = df['addresses'].apply(lambda x: check_urb(x))

def extract_address_components(address):
    """
    This function extracts the street, street number, and floor from the address
    """
    # Define a regular expression pattern to extract the street number, street, and floor
    pattern = r'\b(\d+)\s+(\S+)\s*(.*)'

    if isinstance(address, str):  # Check if the value is a string
        match = re.search(pattern, address)
        if match:
            street_number = match.group(1)
            # street = match.group(2)
            floor = match.group(2) + ' ' + match.group(3)
            # return street, street_number, floor
            return street_number, floor
    # return '', '', ''
    return '', ''

# Apply the function to the 'address' column and create new columns 'street', 'street_number', and 'floor'
df[['street_number', 'floor']] = df['addresses'].apply(extract_address_components).apply(pd.Series)

def extract_street_number(address):
    """  
    This function extracts the street number from the address
    """
    # Define a regular expression pattern to extract the street number
    pattern = r'\b(\d+)(?:\s|$)'
    if isinstance(address, str):  # Check if the value is a string
        match = re.search(pattern, address)
        if match:
            return match.group(1)
    return ''

# Apply the function to the 'address' column and create a new column 'street_number'
df['street_number'] = df['addresses'].apply(extract_street_number)

# Function to extract the street number from the address
def extract_street_number2(address):
    """ 
    This function extracts the street number from the address with a different pattern.
    """
    
    # Define a regular expression pattern to extract the street number
    pattern = r'\b(\d+)\b'
    if isinstance(address, str):  # Check if the value is a string
        matches = re.findall(pattern, address)
        if matches:
            return ' '.join(matches)
    return ''

# Apply the function to the 'address' column and create a new column 'street_number'
df['street_number'] = df.apply(lambda row: extract_street_number2(row['addresses']) if row['street_number'] == '' else row['street_number'], axis=1)

def check_st_num(st_num):
    """ 
    This function is to check if the street number is a valid number and correct it if it is not.
    Ex: 4242 -> 42
    """
    if len(st_num) >= 4:
        n = len(st_num) // 2
        return st_num[:n]
    return st_num

# Apply the check_st_num function to the 'street_number' column    
df['street_number'] = df['street_number'].apply(lambda x: check_st_num(x))

# Convert all addresses to string type
df['addresses'] = df['addresses'].astype(str)

# Extract the street name from the address
# df['street_name'] = df['addresses'].apply(lambda x: ' '.join(re.findall(r'[^\d]+', x.split(',')[0])).strip())

# Check floor_numbers
df['floor_number'] = df['addresses'].apply(lambda x: re.findall(r'(\d+\s*nd)', x))
df['floor_number'] = df['floor_number'].apply(lambda x: x[0] if x else '')

# Function to extract the street name from the address
def extract_street_name(address):
    if isinstance(address, str):  # Check if the value is a string
        if ',' in address:
            # If address contains a comma, extract all characters before the comma
            street_name = address.split(',')[0]
        elif re.search(r'\d', address):
            # If address contains a number, extract all characters before the first number
            street_name = re.split(r'\d', address)[0]
        else:
            # Otherwise, return the whole address as the street name
            street_name = address
        
        return street_name.strip()  # Remove leading/trailing whitespace from the street name
    return ''

# Apply the function to the 'addresses' column and create a new column 'street_name'
df['street_name2'] = df['addresses'].apply(extract_street_name)

def remove_numbers(address):
    """ 
    This function removes all numbers from the address.
    """
    address_without_numbers = re.sub(r'N°|\b\d+\b', '', address)
    return address_without_numbers.strip()

# Remove numbers from the 'street_name' column
df['street_name2'] = df['street_name2'].apply(lambda x: remove_numbers(x))

def remove_nnd(address):
    """ 
    This function removes nnd from the address.
    """
    address_without_nnd = re.sub(r' nnd$| nnd ', '', address, flags=re.IGNORECASE)
    return address_without_nnd.strip()

# Remove nnd from the 'street_name' and 'floor' column
df['street_name2'] = df['street_name2'].apply(lambda x: remove_nnd(x))
df['floor'] = df['floor'].apply(lambda x: remove_nnd(x))

def remove_nd(address):
    """ 
    This function removes nd from the address.
    """
    address_without_nnd = re.sub(r' nd$| nd ', '', address, flags=re.IGNORECASE)
    return address_without_nnd.strip()

# Remove nd from the 'street_name' and 'floor' column
df['street_name2'] = df['street_name2'].apply(lambda x: remove_nd(x))
df['floor'] = df['floor'].apply(lambda x: remove_nd(x))


def remove_parentheses(address):
    """ 
    This function removes parentheses from the address.
    And remove the content inside the parentheses.
    """
    address_without_parentheses = re.sub(r'\((.*)\)', '', address)
    return address_without_parentheses.strip()

# Remove parentheses from the 'street_name' column
df['street_name2'] = df['street_name2'].apply(lambda x: remove_parentheses(x))

df.to_csv('Cleaned_Data.csv')

df

,type_of_street,addresses,neighborhood,province,street_number,floor,floor_number,street_name2
1,calle,san martin de porres 10,vista alegre,murcia,10,,,san martin de porres
2,calle,"collado de los jeronimos, fase 2 , 21 , 2 b...",casa pelada,murcia,2,", 21 , 2 b - la tercia golf",,collado de los jeronimos
3,,avenida. del carbono 107,cartagena,murcia,107,,,avenida. del carbono
4,,urbanizacion portal 1 punta cormoran 44 nd 2,la manga del mar menor,murcia,1,punta cormoran 44 2,44 nd,urbanizacion portal
5,calle,granizo nnd 4242,los camachos,murcia,42,,,granizo
...,...,...,...,...,...,...,...,...
6941,,joan fuster 7 pta 9,xixona,alicante,7,pta 9,,joan fuster
6942,avenida,consell pais valencia 57,agost,alicante,57,,,consell pais valencia
6944,,virgen del remedio 67 3 nd puerta a,novelda,alicante,67,3puerta a,3 nd,virgen del remedio
6945,,virgen de la salud 13,hondon de los frailes,alicante,13,,,virgen de la salud


In [14]:
df.to_csv('test_urban.csv')